# Input

In [175]:
file_path = 'uml.txt'

with open(file_path, 'r') as file:

    uml = file.read()    # Read the file line by line
    # print(uml)


In [176]:
final_uml = uml
for line in uml.splitlines():
    if "//" in line:
        print(line)
        line_no_comment = line.split("//")[0]
        print(line_no_comment)
        # line_no_comment += "\n"

        final_uml = final_uml.replace(line, line_no_comment)

uml = final_uml


{ //adress of chosen home
{ 
{ //unit conaining devices
{ 
{ //abstract class for Sensor and Actuator
{ 
  enum SensorType{mouvement,temperature,other}; //there could be other sensor types that i do not know of
  enum SensorType{mouvement,temperature,other}; 
  enum ActuatorType{light,lock,other}; //there could be other types of actuators that i do not know of
  enum ActuatorType{light,lock,other}; 
  * readings -- 1 Sensor sensorDevice; //readings of 1 sensor device
  * readings -- 1 Sensor sensorDevice; 
	//reflexice associations for complex hierarchy within automation rules
	
  enum CommandType{lockDoor,turnOnHeating,other}; //other types of predefined commands exist
  enum CommandType{lockDoor,turnOnHeating,other}; 
  enum CommandStatus{requested,completed,failed,pending,other}; //there could be other command statuses
  enum CommandStatus{requested,completed,failed,pending,other}; 
  1 preCondition -<@> 1 AutomationRule rule; //automation rule uses precondition
  1 preCondition -<@

In [177]:
final_uml = uml
for line in uml.splitlines():
    if "enum" in line:
        print(line.replace("{","(").replace("}", ")").replace("enum","").strip())
        final_uml = final_uml.replace(line, "")

uml = final_uml

DeviceState(activated,deactivated);
SensorType(mouvement,temperature,other);
ActuatorType(light,lock,other);
State(activated,deactivated);
CommandType(lockDoor,turnOnHeating,other);
CommandStatus(requested,completed,failed,pending,other);
BooleanOps(AND,OR,NOT);


# Parse

## functions

In [178]:
import re

def extract_content_within_braces(input_string):
    # Define a regular expression pattern to match content within curly braces
    pattern = r'\{([^}]+)\}'

    # Use re.findall to find all matches in the input string
    matches = re.findall(pattern, input_string)

    
    # Return the list of matches
    return matches

# Example usage
input_string = "{bla bla} {example} {another example}"
result = extract_content_within_braces(input_string)

print("Original string:", input_string)
print("Content within braces:", result)

Original string: {bla bla} {example} {another example}
Content within braces: ['bla bla', 'example', 'another example']


In [179]:
def create_nested_list(matchs):
    nested_list = []
    for paragraph in matchs:
        true_line = []
        lines = paragraph.split('\n')
        for i in range(len(lines)):
            lines[i] = lines[i].strip()
            if len(lines[i])>0 and not "position" in lines[i]:
                true_line.append(lines[i])

        
        nested_list.append(true_line)

    return nested_list


In [180]:
def find_class(uml):
    ans = []
    lines = uml.splitlines()

    for l in lines:
        if "class" in l:
            cls = l.replace("class","")
            cls = cls.strip()
            if not cls in ans:
                ans.append(cls)
    return ans

In [181]:
def is_rel(line):
    contain = "<@>"
    aso = "--"
    uni_aso_l = "->"
    uni_aso_r = "<-"
    isa = "isA"
    rels = [contain, aso, uni_aso_l, uni_aso_r, isa]

    for rel in rels:
        if rel in line:
            return True
    return False

In [182]:
def process_rel(cls, line):
    line = line.split(";")[0]
    
    contain = "<@>"
    aso = "--"
    uni_aso_l = "->"
    uni_aso_r = "<-"
    isa = "isA"
    if "-<@>" in line:
        elements = line.split("-<@>")
        return elements[1] + " contain " + elements[0] + cls
    if "<@>-" in line:
        elements = line.split("<@>-")
        return elements[0] + cls +  " contain " + elements[1]      

    if "--" in line:
        elements = line.split("--")
        return elements[0] + cls +  " associate " + elements[1]      

    if "->" in line:
        elements = line.split("->")
        return elements[0] + cls +  " associate " + elements[1] 
    
    if "<-" in line:
        elements = line.split("<-")
        return elements[0] + cls +  " associate " + elements[1]      
    
    if isa in line:
        return cls + line.replace("isA", " inherit ")
    
    return "cannot process:" + line

## inference

In [183]:
print(uml)

class SHAS
{
}
class SmartHome
{
  1 home <@>- * Room roomInHome;
  * homes -<@> 1 SHAS system;
}

class Adress 
{ 
  1 adressOf -- 1 SmartHome home;
  street;
  city;
  country;
  int postalCode;
}

class User
{
  * users -<@> 1 SHAS;
}

class Room 
{ 
  1 room <@>- * Device devices;
}

class Device
{ 
  int deviceID;

  DeviceState state;
  abstract;
}

class Sensor
{
  SensorType kind;
  isA Device;

}

class Actuator
{
  ActuatorType kind;
  isA Device;
  1 actuatorDevice -- * ControlCommand controlCommand;

}

class SensorReading
{
  * readings -- 1 Sensor sensorDevice; 
  Double measuredValue;
  Time timeStamp;
}

class AutomationRule
{
  * rule -<@> 1 User user;

    State ruleState;
	
  0..1 rule1 -- * AutomationRule dependantRule;
  0..1 rule2 -- * AutomationRule conflictRule;
}

class ControlCommand
{
  Time timeStamp;


  CommandType commandType; 
  CommandStatus commandStatus;
}

class ActivityLog
{
  1 activityLog -<@> 1 SmartHome;
  1 -- * SensorReading readings;
  1 -- *

In [184]:
classes = find_class(uml)
print(classes)
len(classes)

['SHAS', 'SmartHome', 'Adress', 'User', 'Room', 'Device', 'Sensor', 'Actuator', 'SensorReading', 'AutomationRule', 'ControlCommand', 'ActivityLog', 'PreCondition', 'Action', 'AtomicRelationalTerm', 'BooleanRelationalTerm']


16

In [185]:
result = extract_content_within_braces(uml)
result = create_nested_list(result)
print(result)
len(result)

[[], ['1 home <@>- * Room roomInHome;', '* homes -<@> 1 SHAS system;'], ['1 adressOf -- 1 SmartHome home;', 'street;', 'city;', 'country;', 'int postalCode;'], ['* users -<@> 1 SHAS;'], ['1 room <@>- * Device devices;'], ['int deviceID;', 'DeviceState state;', 'abstract;'], ['SensorType kind;', 'isA Device;'], ['ActuatorType kind;', 'isA Device;', '1 actuatorDevice -- * ControlCommand controlCommand;'], ['* readings -- 1 Sensor sensorDevice;', 'Double measuredValue;', 'Time timeStamp;'], ['* rule -<@> 1 User user;', 'State ruleState;', '0..1 rule1 -- * AutomationRule dependantRule;', '0..1 rule2 -- * AutomationRule conflictRule;'], ['Time timeStamp;', 'CommandType commandType;', 'CommandStatus commandStatus;'], ['1 activityLog -<@> 1 SmartHome;', '1 -- * SensorReading readings;', '1 -- * ControlCommand commands;'], ['1 preCondition -<@> 1 AutomationRule rule;'], ['1 action -<@> 1 AutomationRule rule;', '0..1 sequenceOf -> * ControlCommand commands;'], ['Room room;', 'Sensor sensor;', '

16

In [190]:
for cls, atr in zip(classes, result):
    print(cls)
    cls_line = cls + "("
    has_atr = False
    for a in atr:
        print(a)
        
        if not is_rel(a):
            a = a.replace(";", ",")
            cls_line = cls_line + " " + a
            has_atr = True
        else:
            print("-"*10 + " rel below " + "-"*10)
            print(process_rel(cls, a).strip())
            print(" ")

    if has_atr:
        cls_line = cls_line[:-1]
    cls_line += ")"
    print(cls_line)
    print()
    print( "=" * 40)

SHAS
SHAS()

SmartHome
1 home <@>- * Room roomInHome;
---------- rel below ----------
1 home SmartHome contain  * Room roomInHome
 
* homes -<@> 1 SHAS system;
---------- rel below ----------
1 SHAS system contain * homes SmartHome
 
SmartHome()

Adress
1 adressOf -- 1 SmartHome home;
---------- rel below ----------
1 adressOf Adress associate  1 SmartHome home
 
street;
city;
country;
int postalCode;
Adress( street, city, country, int postalCode)

User
* users -<@> 1 SHAS;
---------- rel below ----------
1 SHAS contain * users User
 
User()

Room
1 room <@>- * Device devices;
---------- rel below ----------
1 room Room contain  * Device devices
 
Room()

Device
int deviceID;
DeviceState state;
abstract;
Device( int deviceID, DeviceState state, abstract)

Sensor
SensorType kind;
isA Device;
---------- rel below ----------
Sensor inherit  Device
 
Sensor( SensorType kind)

Actuator
ActuatorType kind;
isA Device;
---------- rel below ----------
Actuator inherit  Device
 
1 actuatorDevice